# Clumping by calling PLINK

In [1]:
import sys
sys.path.insert(0,"/home/yunye/work/gwaslab/src")
import gwaslab as gl

In [2]:
gl.show_version()

2024/12/23 16:17:45 GWASLab v3.5.4 https://cloufield.github.io/gwaslab/
2024/12/23 16:17:45 (C) 2022-2024, Yunye He, Kamatani Lab, MIT License, gwaslab@gmail.com


# Load sample data and perform QC

In [3]:
mysumstats = gl.Sumstats("./0_sample_data/t2d_bbj.txt.gz",
             snpid="SNP",
             chrom="CHR",
             pos="POS",
             se="SE",
             p="P",nrows=1000000)

2024/12/23 16:17:45 GWASLab v3.5.4 https://cloufield.github.io/gwaslab/
2024/12/23 16:17:45 (C) 2022-2024, Yunye He, Kamatani Lab, MIT License, gwaslab@gmail.com
2024/12/23 16:17:45 Start to initialize gl.Sumstats from file :./0_sample_data/t2d_bbj.txt.gz
2024/12/23 16:17:46  -Reading columns          : CHR,SNP,P,SE,POS
2024/12/23 16:17:46  -Renaming columns to      : CHR,SNPID,P,SE,POS
2024/12/23 16:17:46  -Current Dataframe shape : 1000000  x  5
2024/12/23 16:17:46  -Initiating a status column: STATUS ...
2024/12/23 16:17:46  #WARNING! Version of genomic coordinates is unknown...
2024/12/23 16:17:46 Start to reorder the columns...v3.5.4
2024/12/23 16:17:46  -Current Dataframe shape : 1000000 x 6 ; Memory usage: 63.43 MB
2024/12/23 16:17:46  -Reordering columns to    : SNPID,CHR,POS,SE,P,STATUS
2024/12/23 16:17:46 Finished reordering the columns.
2024/12/23 16:17:46  -Column  : SNPID  CHR    POS   SE      P       STATUS  
2024/12/23 16:17:46  -DType   : object string int64 float64 flo

In [4]:
mysumstats.fix_id(fixsep=True)

2024/12/23 16:17:46 Start to check SNPID/rsID...v3.5.4
2024/12/23 16:17:46  -Current Dataframe shape : 1000000 x 6 ; Memory usage: 63.43 MB
2024/12/23 16:17:46  -Checking SNPID data type...
2024/12/23 16:17:46  -Converting SNPID to pd.string data type...
2024/12/23 16:17:46  -Checking if SNPID is CHR:POS:NEA:EA...(separator: - ,: , _)
2024/12/23 16:17:48  -Replacing [_-] in SNPID with ":" ...
2024/12/23 16:17:49 Finished checking SNPID/rsID.


In [5]:
mysumstats.basic_check(verbose=False)

2024/12/23 16:17:54  #WARNING! Necessary columns for .fix_allele() were not detected:EA,NEA
2024/12/23 16:17:55  #WARNING! Necessary columns for .normalize() were not detected:EA,NEA


## Run clumping by calling plink2

GWASLab will extract lead variants and prepare input files dor running clumping by calling PLINK2

- vcf: reference LD VCF file (it will be converted to bfile)
- bfile: PLINK bfile prefix
- pfile: PLINK pfile prefix

Default parameters:
    
- clump_p1=5e-8
- clump_p2=5e-8
- clump_r2=0.01
- clump_kb=250

In [6]:
mysumstats.clump(   clump_r2=0.1,
                    clump_p2=1e-6,
                    vcf=gl.get_path("1kg_eas_hg19"))

2024/12/23 16:17:55 Start to perfrom clumping...v3.5.4
2024/12/23 16:17:55  -Current Dataframe shape : 1000000 x 6 ; Memory usage: 65.33 MB
2024/12/23 16:17:55 Start to perform clumping...
2024/12/23 16:17:55  -Clumping parameters for PLINK2:
2024/12/23 16:17:55   -clump_p1 : 5e-08...
2024/12/23 16:17:55   -clump_p2 : 1e-06...
2024/12/23 16:17:55   -clump_kb : 250...
2024/12/23 16:17:55   -clump_r2 : 0.1...
2024/12/23 16:17:55  -Clumping will be performed using P
2024/12/23 16:17:55  -Significant variants on CHR:  [1, 2]
2024/12/23 16:17:55  -Processing VCF : /home/yunye/.gwaslab/EAS.ALL.split_norm_af.1kgp3v5.hg19.vcf.gz...
2024/12/23 16:17:56  -PLINK version: PLINK v2.00a5.9LM AVX2 AMD (12 Dec 2023)
2024/12/23 16:17:56   -Processing VCF for CHR 1...
2024/12/23 16:17:56   -Plink bfile for CHR 1 exists: /home/yunye/.gwaslab/EAS.ALL.split_norm_af.1kgp3v5.hg19.1. Skipping...
2024/12/23 16:17:56   -Processing VCF for CHR 2...
2024/12/23 16:17:56   -Plink bfile for CHR 2 exists: /home/yunye

### clump results are stored in mysumstats.clumps

In [8]:
mysumstats.clumps["clumps"]

,SNPID,CHR,POS,SE,P,STATUS
96739,1:22068326:A:G,1,22068326,0.0103,1.629000e-09,9960999
213860,1:51103268:T:C,1,51103268,0.0120,2.519000e-11,9960999
214500,1:51401146:CT:C,1,51401146,0.0145,6.090000e-10,9960999
534095,1:154309595:TA:T,1,154309595,0.0166,3.289000e-08,9960999
969974,2:640986:CACAT:C,2,640986,0.0150,2.665000e-10,9960999


In [9]:
mysumstats.clumps["clumps_raw"]

,CHR,POS,SNPID,P,TOTAL,NONSIG,S0.05,S0.01,S0.001,S0.0001,SP2
2,1,22068326,1:22068326:A:G,1.629000e-09,89,0,0,0,0,89,"1:21982702:A:G,1:21983254:C:T,1:21983611:C:T,1..."
0,1,51103268,1:51103268:T:C,2.519000e-11,347,0,0,0,0,347,"1:50854197:G:C,1:50854654:A:G,1:50855083:A:G,1..."
1,1,51401146,1:51401146:CT:C,6.090000e-10,134,0,0,0,0,134,"1:51353720:C:G,1:51355823:A:G,1:51356091:G:T,1..."
3,1,154309595,1:154309595:TA:T,3.289000e-08,5,0,0,0,0,5,"1:154264194:G:A,1:154321382:C:T,1:154345344:G:..."
4,2,640986,2:640986:CACAT:C,2.665000e-10,232,0,0,0,0,232,"2:601905:T:G,2:605176:T:A,2:609177:A:G,2:61060..."


In [11]:
print(mysumstats.clumps["plink_log"])

PLINK v2.00a5.9LM AVX2 AMD (12 Dec 2023)       www.cog-genomics.org/plink/2.0/
(C) 2005-2023 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to clumping_plink2.1.log.
Options in effect:
  --bfile /home/yunye/.gwaslab/EAS.ALL.split_norm_af.1kgp3v5.hg19.1
  --chr 1
  --clump _gwaslab_tmp.1.SNPIDP
  --clump-id-field SNPID
  --clump-kb 250
  --clump-p-field P
  --clump-p1 5e-08
  --clump-p2 1e-06
  --clump-r2 0.1
  --out clumping_plink2.1
  --threads 1

Start time: Mon Dec 23 16:18:04 2024
31934 MiB RAM detected, ~27652 available; reserving 15967 MiB for main
workspace.
Using 1 compute thread.
504 samples (0 females, 0 males, 504 ambiguous; 504 founders) loaded from
/home/yunye/.gwaslab/EAS.ALL.split_norm_af.1kgp3v5.hg19.1.fam.
6471908 variants loaded from
/home/yunye/.gwaslab/EAS.ALL.split_norm_af.1kgp3v5.hg19.1.bim.
Note: No phenotype data present.

--clump: 0/312 index candidates processed.
--clump: 4 clumps formed from 312 index candidates.  
Results written to